In [ ]:
# 비율별로 데이터셋 추출출
from datasets import load_dataset
import json
import random
from pathlib import Path

# 🔧 설정: 데이터셋 목록 및 비율
datasets_config = {
    "koalpaca": {
        "path": "beomi/KoAlpaca-v1.1a",
        "split": "train",
        "ratio": 0.50
    },
    "openorca": {
        "path": "kyujinpy/KOR-OpenOrca-Platypus-v3",
        "split": "train",
        "ratio": 0.25
    },
    "kullm": {
        "path": "nlpai-lab/kullm-v2",
        "split": "train",
        "ratio": 0.15
    },
    "sharegpt": {
        "path": "FreedomIntelligence/sharegpt-korean",
        "split": "train",
        "ratio": 0.10
    }
}

total_samples = 100_000
merged_data = []

# 📌 KoAlpaca 포맷 변환 함수
def convert_to_koalpaca_format(name, entry):
    if name == "sharegpt":
        conversations = entry.get("conversations", [])
        if len(conversations) >= 2:
            return {
                "instruction": conversations[0].get("value", "").strip(),
                "input": "",
                "output": conversations[1].get("value", "").strip()
            }
    else:
        instruction = entry.get("instruction") or entry.get("prompt", "")
        output = entry.get("output") or entry.get("completion", "")
        if instruction and output:
            return {
                "instruction": instruction.strip(),
                "input": "",
                "output": output.strip()
            }
    return None

# 📦 데이터 로딩 및 변환
for name, config in datasets_config.items():
    print(f"🔄 Loading: {name} from {config['path']}")
    dataset = load_dataset(config["path"], split=config["split"])
    sample_count = int(total_samples * config["ratio"])
    sampled = dataset.shuffle(seed=42).select(range(min(sample_count, len(dataset))))

    for entry in sampled:
        converted = convert_to_koalpaca_format(name, entry)
        if converted:
            merged_data.append(converted)

# 💾 저장
output_path = Path("../data/merged_ko_instruction_100k.jsonl")
with open(output_path, "w", encoding="utf-8") as f:
    for item in merged_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ 저장 완료: {output_path} (총 샘플 수: {len(merged_data)})")


🔄 Loading: koalpaca from beomi/KoAlpaca-v1.1a
🔄 Loading: openorca from kyujinpy/KOR-OpenOrca-Platypus-v3
🔄 Loading: kullm from nlpai-lab/kullm-v2
🔄 Loading: sharegpt from FreedomIntelligence/sharegpt-korean


README.md:   0%|          | 0.00/204 [00:00<?, ?B/s]

sharegpt-korean.json:   0%|          | 0.00/33.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6014 [00:00<?, ? examples/s]

✅ 저장 완료: ../data/merged_ko_instruction_100k.jsonl (총 샘플 수: 67062)


In [ ]:
# 100 K 샘플로 무작위 추출
from datasets import load_dataset
import json
import random
from pathlib import Path

# 🔧 사용할 한국어 데이터셋
datasets_config = {
    "koalpaca": {
        "path": "beomi/KoAlpaca-v1.1a",
        "split": "train"
    },
    "openorca": {
        "path": "kyujinpy/KOR-OpenOrca-Platypus-v3",
        "split": "train"
    },
    "kullm": {
        "path": "nlpai-lab/kullm-v2",
        "split": "train"
    },
    "sharegpt": {
        "path": "FreedomIntelligence/sharegpt-korean",
        "split": "train"
    }
}

merged_data = []
all_valid_entries = []

# ✅ 변환 함수 (KoAlpaca 포맷 통일)
def convert_to_koalpaca_format(name, entry):
    if name == "sharegpt":
        conversations = entry.get("conversations", [])
        if len(conversations) >= 2:
            return {
                "instruction": conversations[0].get("value", "").strip(),
                "input": "",
                "output": conversations[1].get("value", "").strip()
            }
    else:
        instruction = entry.get("instruction") or entry.get("prompt", "")
        output = entry.get("output") or entry.get("completion", "")
        if instruction and output:
            return {
                "instruction": instruction.strip(),
                "input": "",
                "output": output.strip()
            }
    return None

# ✅ 모든 유효 데이터 모으기
for name, config in datasets_config.items():
    print(f"🔄 Loading: {name} from {config['path']}")
    dataset = load_dataset(config["path"], split=config["split"])
    dataset = dataset.shuffle(seed=42)

    valid = []
    for entry in dataset:
        item = convert_to_koalpaca_format(name, entry)
        if item:
            valid.append(item)

    print(f"✅ {name} usable: {len(valid)}")
    all_valid_entries.extend(valid)

# ✅ 100K로 무작위 추출
random.seed(42)
final_dataset = random.sample(all_valid_entries, min(100_000, len(all_valid_entries)))

# 💾 저장
output_path = Path("../data/merged_ko_instruction_100k.jsonl")
with open(output_path, "w", encoding="utf-8") as f:
    for item in final_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ 최종 저장 완료: {output_path} (총 샘플 수: {len(final_dataset)})")


🔄 Loading: koalpaca from beomi/KoAlpaca-v1.1a
✅ koalpaca usable: 21155
🔄 Loading: openorca from kyujinpy/KOR-OpenOrca-Platypus-v3
✅ openorca usable: 34212
🔄 Loading: kullm from nlpai-lab/kullm-v2
✅ kullm usable: 151346
🔄 Loading: sharegpt from FreedomIntelligence/sharegpt-korean
✅ sharegpt usable: 6011
✅ 최종 저장 완료: ../data/merged_ko_instruction_100k.jsonl (총 샘플 수: 100000)


In [12]:
import os

file_path = "../data/merged_ko_instruction_100k.jsonl"

if os.path.exists(file_path):
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"📦 파일 용량: {size_mb:.2f} MB")
else:
    print("❌ 파일이 존재하지 않습니다.")


📦 파일 용량: 154.44 MB


In [13]:
file_path = "../data/merged_ko_instruction_100k.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    line_count = sum(1 for _ in f)

print(f"📊 총 샘플 수: {line_count}")


📊 총 샘플 수: 100000


In [15]:
jsonl_path = "../data/merged_ko_instruction_100k.jsonl"
raw_dataset = load_dataset("json", data_files=jsonl_path, split="train")
split_dataset = raw_dataset.train_test_split(test_size=0.05, seed=42)
train_raw = split_dataset["train"]
val_raw = split_dataset["test"]

Generating train split: 0 examples [00:00, ? examples/s]

['해바라기씨를 과다 복용하면 문제가 될까요? 그리고 임산부는 왜 많이 못 먹게 되는 걸까요? \n사람들이 해바라기씨를 적당한 양으로 먹을 때는 건강에 매우 좋다는데, 과다 복용을 하면 살이 찌거나 얼굴이 번들거리는 등 부작용이 있을까요? 그리고 임산부들은 해바라기씨를 왜 많이 못 먹게 되는 걸까요?',
 '5줄의 예제를 주세요',
 '오늘의 비트코인 가격을 미국 달러로 구합니다.',
 '그렇다면 전체 여행에 드는 총 유류비는 얼마일까요?',
 '"상태"를 "상태"로 변환하는 데 필요한 SQL 트리를 만들 수 있습니다.',
 '식빵을 냉장고에 넣으면 안되는 이유는 무엇인가요? 그렇다면 어떤 방법으로 식빵을 보관해야 하나요?',
 '맹독성 어종이나 파충류의 색깔이 화려한 이유는 무엇일까요?',
 '계속 생성',
 '아래 그림에서 점 $A$, $B$, $C$, $P$는 $PA=2$, $PB=3$, $PC=4$, $BC=5$가 되도록 배치되어 있습니다. 삼각형 ABC$의 가능한 최대 면적은 얼마입니까? [asy]\ndefaultpen(linewidth(0.8)); size(150);\n쌍 B = (0,0), C = (5,0), A = (2,3), P = (2.2,2);\ndraw(A--B--C--cycle^^B--P^^C--P^^A--P);\nlabel("$A$",A,N); label("$B$",B,S); label("$C$",C,S); label("$P$",P,S);\n[/asy]',
 '변호사가 이러한 각 하위 주제에 대한 기사를 작성하려면 어떻게 해야 하나요? 각각의 법적 주제는 무엇인가요?',
 '숫자 $r$은 4자리 소수 $0.abcd,$로 표현할 수 있으며, 여기서 $a, b, c 및 $d$는 숫자를 나타내며, 이 중 어느 하나라도 0이 될 수 있습니다. 분자가 1 또는 2이고 분모가 정수인 분수로 $r$을 근사화하고자 합니다. r$에 가장 가까운 분수는 $\\frac 27$입니다. $r$의 가능한 값의 수는 얼마입니까?',
 '교통 혼잡을 줄이기 위해 